# Create and Train ANN




In [1]:
# Run this cell first
import os
import sys
RunningInCOLAB  = 'google.colab' in str(get_ipython())

if RunningInCOLAB:
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    DIRECTORY = '/content/drive/My Drive/AMN-Colab/'
    sys.path.append('/content/drive/My Drive/amn')
    ! pip install cobra
    ! pip install silence_tensorflow
else:
    DIRECTORY = './'
print(os.listdir(DIRECTORY))
    
from Build_Model import *

def printout(filename, Stats, model, time): 
    # printing Stats
    if Stats == None:
        print('Stats for %s failed CPU-time %.4f' % (filename, time))
        return
    print('Stats for %s CPU-time %.4f' % (filename, time))
    print('R2 = %.4f (+/- %.4f) Constraint = %.4f (+/- %.4f)' % \
          (Stats.train_objective[0], Stats.train_objective[1],
           Stats.train_loss[0], Stats.train_loss[1]))
    print('Q2 = %.4f (+/- %.4f) Constraint = %.4f (+/- %.4f)' % \
          (Stats.test_objective[0], Stats.test_objective[1],
           Stats.test_loss[0], Stats.test_loss[1]))

['Result', '.DS_Store', 'Dataset_model', 'Reservoir', 'Build_Mode_Dense.ipynb', 'old', '__pycache__', 'README.md', '.gitignore', 'Build_Dataset.ipynb', 'Build_Model.py', '.ipynb_checkpoints', 'Dataset_input', 'Build_Dataset.py', '.git', 'Build_Model.ipynb']


In [ ]:
# Create, train and evaluate ANN models with FBA simulated training set for E. coli core

# What you can change 
seed = 10
np.random.seed(seed=seed)  
trainname = 'e_coli_core_UB_10000'
size = 1000
# End of What you can change

# Load training set
trainingfile = DIRECTORY+'Dataset_model/'+trainname
cobramodel = cobra.io.read_sbml_model(trainingfile+'.xml')
OBJ = [r.id for r in cobramodel.reactions]
parameter = TrainingSet()
parameter.load(trainingfile)

# Set all pred fluxes to zero
Y, zeros = {}, 0 * parameter.Y 
for i in range(len(OBJ)):
    Y[i] = zeros[:,0] # zero vector

# train for different flux outputs 
# when size > len(OBJ) all flux are taken one after another 
# otherwise (small trainig sets) output fluxes are chosen at random
size = size if size < len(OBJ) else len(OBJ)
for k in range(size):
    
    # choose an output flux at random when size < range(len(OBJ))
    i = k if size == len(OBJ) else np.random.randint(0,high=len(OBJ))
    objective = [OBJ[i]]
    
    # create model
    print(k+1,'/',size, objective,'----------------------------------------------------------------------')
    if np.sum(parameter.Y[:,i]) == 0:
        continue # Skip all parameter.Y beeing at zero
    model = Neural_Model(trainingfile = trainingfile, 
                         objective=objective, 
                         model_type = 'ANN_Dense',
                         n_hidden = 1, hidden_dim = 50, 
                         epochs = 500, xfold = 5)
        
    # Train and evaluate
    start_time = time.time()
    try:
        reservoir, pred, stats, _ = train_evaluate_model(model, verbose=False)
    except:
        reservoir, pred, stats, _ = None, zeros, None, None
    delta_time = time.time() - start_time

    # Printing cross-validation results
    printout(objective, stats, reservoir, delta_time)
    Y[i] = pred[:,0]

# Collate all predicted Y and get stats and constraints
Y = np.transpose(np.asarray(list(Y.values())))
print(Y.shape, parameter.Y.shape)
print('Q2=', r2_score(parameter.Y, Y, multioutput='variance_weighted'))
X = tf.convert_to_tensor(np.float32(model.X)) # Loss computed of tf tensors
Y = tf.convert_to_tensor(np.float32(Y))
L2, _ = Loss_SV(Y, model.S)
L2 = np.mean(L2.numpy())
print('Loss_SV =', L2)
L3, _ = Loss_Vin(Y, model.Pin, X, 'UB')
L3 = np.mean(L3.numpy())
print('Loss_Vin =', L3)
L = (L2+L3)/2
print('Constraints =', L)

1 / 154 ['PFK'] ----------------------------------------------------------------------
nbr parameters: 1101
